In [1]:
import pandas as pd
import json
import numpy as np

In [298]:
# Countries (NOCs)
with open('data/medals.json') as f:
    data = json.load(f)
    data = data['props']['pageProps']

df = pd.DataFrame(data['nocList'])
df = df.drop(columns=["nameOrder", "longNameOrder"])
df = df.rename(columns={"id": "country_id", "name": "country_name"})

df.to_csv('data/countries.csv', index=False)

In [299]:
# Disciplines
with open('data/medals.json') as f:
    data = json.load(f)
    data = data['props']['pageProps']

rows = []
for discipline in data['initialFilterDisciplines']['disciplines']:
    rows.append({
        "discipline_id": discipline['id'],
        "discipline_name": discipline['name'],
        "eventList": [e['id'].rstrip('-') for e in discipline['events']]
    })

df = pd.DataFrame(rows)
df.to_csv('data/disciplines.csv', index=False)

In [300]:
# Events
with open('data/medals.json') as f:
    data = json.load(f)
    data = data['props']['pageProps']

rows = []
for discipline in data['initialFilterDisciplines']['disciplines']:
    for event in discipline['events']:
        rows.append({
            "event_id": event['id'].rstrip('-'),
            "event_name": event['name'],
            "discipline_id": discipline['id']
        })

df = pd.DataFrame(rows)
df.to_csv('data/events.csv', index=False)

In [12]:
# Athletes
with open('data/athletes.json') as f:
    data = json.load(f)
    athletes = data['persons']

rows = []

for athlete in athletes:
    row = {
        "athlete_id": athlete['code'],
        "name": athlete['name'],
        "tvName": athlete['TVName'],
        "dob": athlete['birthDate'],
        "gender": athlete['personGender']['description'],
        "role": athlete['mainFunction']['description'],
        "country_id": athlete['organisation']['code'],
        "disciplineList": [d['code'] for d in athlete['disciplines']],
        "eventList": [e['event']['code'].rstrip('-') for e in athlete['registeredEvents']]
    }
    rows.append(row)

df = pd.DataFrame(rows)
df.to_csv('data/athletes.csv', index=False)

In [306]:
# Medals
# Every single medal won (athlete level, not team level – e.g., there's a medal for all members of a hockey team)

# Hence, the following line will recreate the final medal tally:
# df.drop_duplicates(subset=['country_id', 'event_id', 'medal_type']).groupby(['country_id']).size().sort_values(ascending=False)

with open('data/medallists.json') as f:
    data = json.load(f)
    medallists = data['props']['pageProps']['initialMedallist']['athletes']

rows = []

for medallist in medallists:
    for medal in medallist['medals']:
        rows.append({
            "athlete_id": medallist['code'],
            "country_id": medallist['organisation'],
            "event_id": medal['event'].rstrip('-'),
            "discipline_id": medal['disciplineCode'],
            "date": medal['date'],
            "medal_type": medal['medalType']
        })

mdf = pd.DataFrame(rows)
mdf.to_csv('data/medallists.csv', index=False)

In [16]:
# Medal tally -- cen be recreated from medals.csv but also available directly
with open('data/medals.json') as f:
    data = json.load(f)
    data = data['props']['pageProps']

standings = data['initialMedals']['medalStandings']['medalsTable']

medals_rows = []

for country in standings:
    
    # Compute the disciplines and events won by the country
    disciplines_won = []
    events_won = []
    for discipline in country['disciplines']:
        disciplines_won.append(discipline['code'])
        for event in discipline['medalWinners']:
            events_won.append(event['eventCode'].rstrip('-'))

    # Find the index of the total medals (others are Men, Women, Open, Mixed)
    index_with_total = next((i for i, x in enumerate(country['medalsNumber']) if x['type'] == 'Total'), None)
    medals_rows.append({
        'country_id': country['organisation'],
        'rank': country['rank'],
        'gold': country['medalsNumber'][index_with_total]['gold'],
        'silver': country['medalsNumber'][index_with_total]['silver'],
        'bronze': country['medalsNumber'][index_with_total]['bronze'],
        'total': country['medalsNumber'][index_with_total]['total'],
        'unique_disciplines_won': len(set(disciplines_won)),
        'unique_events_won': len(set(events_won)),
    })

df = pd.DataFrame(medals_rows)
df.to_csv('data/medal_table.csv', index=False)

In [2]:
# Bonus: Horses

with open('data/horses.json') as f:
    data = json.load(f)
    horses = data['horses']

rows = []

for horse in horses:

    if 'organisation' not in horse:
        continue

    row = {
        "horse_id": horse['code'],
        "name": horse['name'],
        "short_name": horse['shortName'],
        "year_of_birth": horse['yearBirth'],
        "country_id": horse['organisation']['code'],
        "discipline_id": horse['discipline']['code']
    }
    rows.append(row)

df = pd.DataFrame(rows)
df.to_csv('data/horses.csv', index=False)

# Note: You can get more info about each horse from the API by calling:
# https://olympics.com/OG2024/data/MIS_Horse~comp=OG2024~horse=H000485~lang=ENG.json
# Replace H000485 with the horse code from the df above.